# ANPR PyTesseract 



###  PERSPECTIVE DESKEWING THROUGH CONTOURS 


In [ ]:
import sys
import numpy as np
import cv2
from imutils import contours
import glob

count = 1
for i in glob.glob("path*.jpg"):
    print("New image")
    im = cv2.imread(i)
    shape = im.shape
    print(shape)
    if (im.shape[0] < 50) or (im.shape[1] < 50):
        print("Original image has been resized to 100*100")
        im = cv2.resize(im, dsize =(100, 100), interpolation = cv2.INTER_AREA)
    im3 = im.copy()
    gray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray,(5,5),0)
    thresh = cv2.adaptiveThreshold(blur,255,1,1,11,2)
    
    #################      Now finding Contours         ###################
    cont,hierarchy = cv2.findContours(thresh,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
    # sorted_ctrs = sorted(cont, key=lambda ctr: cv2.boundingRect(ctr)[0])
    '''if parameter is 0, it will sort it on the basis of x axis, 
    else if its 1 then y axis.'''
    # ltr = sorted(cont, key=lambda ctr: cv2.boundingRect(ctr)[0])
    # ltr = sorted(cont, key=lambda ctr: cv2.boundingRect(ctr)
                  # [0] + cv2.boundingRect(ctr)[1] * im.shape[1] )
    ltr = contours.sort_contours(cont, method="left-to-right")[0]#''
    area_list = []
    coord_left= []
    coord_right=[]
    mylist = None
    for cnt in ltr:
        area =  cv2.contourArea(cnt)
        if cv2.contourArea(cnt)>200:
            [x,y,w,h] = cv2.boundingRect(cnt)
            if mylist is None:
                if h>25 and h<70 and w <50:
                    area_list.append(area)
                    mylist = [{
                    'x':x,
                    'y':y,
                    'w':w,
                    'h':h
                }]                 
            else:                    
                if h>25 and h<70 and w <50:
                    mylist.append({'x':x, 'y':y,'w':w, 'h':h})
                    area_list.append(area)
                    print(h)
    if mylist:                
        unique_list = [i for n, i in enumerate(mylist) if i not in mylist[n + 1:]] 
        # break
        for i in unique_list:        
            x = i['x']
            y = i['y']
            w = i['w']
            h = i['h']        
            # cv2.rectangle(im,(x,y),(x+w,y+h),(255,255,255),0)
            # print(x,y,w,h)
            left_top = (x , y)
            right_bottom = (x+w , y+h)
            left_bottom = (x+w , y)
            right_top = (x , y+h)        
            # print("the type iss",type(left_top))
            # print(left_top ,
            #       right_bottom,
            #       left_bottom , 
            #       right_top)
            coord_left.append(left_top)
            coord_right.append(right_bottom)
            print(coord_left)
            # image = cv2.circle(im, (left_top[0],left_top[1]), radius=1, color=(0, 0, 255), thickness=5)
            # image = cv2.circle(im, (right_top[0],right_top[1]), radius=1, color=(255, 0, 0), thickness=5)
            # image = cv2.circle(im, (right_bottom[0],right_bottom[1]), radius=1, color=( 0, 0,255), thickness=5)
            # image = cv2.circle(im, (left_bottom[0],left_bottom[1]), radius=1, color=( 255,0, 0), thickness=5)
            # roi = thresh[y:y+h,x:x+w]
            # roismall = cv2.resize(roi,(10,10))
            # count += 1
            roi = im[y-1:y+h,x-2:x+w+2]
            # cv2.imshow('norm',im)
            # cv2.waitKey(0)
            # roi_resize = cv2.resize(roi, dsize =(50, 50), interpolation = cv2.INTER_AREA)
            # cv2.imwrite('./crop/'+str(count)+'.jpg', roi_resize)
            print("Images are being saved")
                # break
            # cv2.imshow('norm',im)
            # cv2.waitKey(0)
            # cv2.destroyAllWindows()  
    # if len(coord_left)>1:
    if mylist:    
        textleft = coord_left[0]
        textbottom = coord_right[-1]
        x1 = textleft[0]-10
        y1 = textleft[1]-15
        x2 = textbottom[0]+10
        y2 = textbottom[1]+20
        # cv2.rectangle(im,(x1,y1),(x2,y2),(0,255,0),2)
        # cv2.imshow('norm',im)
        # cv2.waitKey(0)
        
        posList= ((x1,y1),(x2,y1-10),(x1-15,y2),(x2-15,y2-10))
        print("Image is being sent for transformation")
        print(posList)
        # specify desired output size 
        width = im.shape[1]
        height = im.shape[0]
        # specify conjugate x,y coordinates (not y,x)
        input1 = np.float32([posList[0], posList[1],posList[2],posList[3]]) #ww323.jpg
        output = np.float32([[0,0], [width,0], [0,height], [width,height]])
        # compute perspective matrix
        matrix = cv2.getPerspectiveTransform(input1,output)
        #do perspective transformation setting area outside input to black
        imgOutput = cv2.warpPerspective(im, matrix, (width,height))#, cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=(0,0,0)
        print(imgOutput.shape)
        # save the warped output
        cv2.imwrite("path"+str(count)+".jpg", imgOutput)
        count += 1
        print("End")
        # cv2.imwrite("PNP_deskewed\\deskewed"+i, imgOutput)
        # show the result
        # cv2.imshow("result", imgOutput)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows() 


###  Pytesseract after Image Enhancment Techniques

In [ ]:
import glob
from collections import Counter
from PIL import Image
import pytesseract
import argparse
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
import re
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'  # your path may be different


lis = []
new= []
for i in glob.glob("path*.jpg"):
#i = '8.jpg'
    print("-------------------- new image ------------------")
    img = cv2.imread(i,0)
    print("Original image size",img.shape)
    img1 = cv2.resize(img, None, fx=6, fy=4)
    bimg = cv2.GaussianBlur(img1,(3,3),0)
    ret,thresh = cv2.threshold(bimg, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    sharpen_filter = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
    sharped_img = cv2.filter2D(thresh, -1, sharpen_filter)
    # Testing all PSM values
    for i in range(6,14):
        #print(f'PSM: [{i}]',pytesseract.image_to_string(sharped_img, 
          #                                config = f'--psm {i} --oem 3 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'))
        text = pytesseract.image_to_string(sharped_img, 
                                      config = f'--psm {i} --oem 3 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789')    
        lis.append(text)
        if lis[0] == '\x0c':
            lis = []    
        else:
            new.append(lis[0].split('\n')[0])   
            lis = []
        plt.subplot(1,1,1),plt.imshow(sharped_img, cmap = 'gray')
        plt.xticks([]),plt.yticks([])
    
    counts = Counter(new).most_common()
    print(counts)
    if len(counts) > 1:
        print("THE EXTRACTED TEXT IS :",counts[0][0])
        new = []

        plt.show()
            #plt.imshow(sharped_img, cmap= 'gray');
        length = len(counts[0][0])
    #print(length)
        if length < 7:
            if length % 2 == 0:
                a_list = list(counts[0][0][0:int(length/2)])
                n_list = list(counts[0][0][int(length/2):])
            else:
                length += 1
                length = int(length/2)
                print(length)
                a_list = list(counts[0][0][0:length])
                n_list = list(counts[0][0][(length)-1:])

        else:    
            num = re.search("[A-Z]{2,4}[0-9]{2,3}",counts[0][0])  
            numerals = re.search("[0-9]{4}", counts[0][0]) 
            if num is not None:
                num = num[0]
            if numerals is not None:
                num_length = len(numerals[0])
                numerals = numerals[0]
                print("NUMERALS CASE CHECK ",numerals)
                if length % 2 == 0:
                    a_list = list(num[0:int(length/2)])
                    n_list = list(numerals[int(length/2):])
                else:
                    length += 1
                    length = int(length/2)
                    a_list = list(num[0:length-1])
                    n_list = list(numerals[-4:])
            elif num is not None:
                #print(num)
                if length % 2 == 0:
                    a_list = list(num[0:int(length/2)])
                    n_list = list(num[int(length/2):])
                else:
                    length += 1
                    length = int(length/2)
                    print(length)
                    a_list = list(num[0:length])
                    n_list = list(num[-3:])    
            #a_list = a_list[-3:]
        #print(a_list)
        #print(n_list)
        ALPHA_LIST = ['A' if i=='4' else 
                   'B' if i=='8' else 
                   'D' if i=='0' else
                   'I' if i=='1' else 
                   'T' if i=='9' else 
                   'S' if i=='5' else
                   'Z' if i== '2' else 
                      i for i in a_list] 
        NUM_LIST = ['4' if i=='A' else 
                   '8' if i=='B' else 
                   '0' if i=='D' else
                   '1' if i=='I' else
                   '9' if i=='T' else 
                   '5' if i=='S' else 
                   '2' if i== 'Z' else i for i in n_list] 
        #print(ALPHA_LIST)
        #print(NUM_LIST)
        if n_list[0] != NUM_LIST[0]:
            ALPHA_LIST = ALPHA_LIST[0:3]
        else:
            ALPHA_LIST = ALPHA_LIST[-3:]    

        FULL_LIST = ALPHA_LIST+NUM_LIST    
        FULL_LIST = ''.join(FULL_LIST)
        
        print("POST PROCESSED EXTRACTED TEXT :",FULL_LIST)   
        FULL_LIST = []

#### assembling both snippets together 






In [ ]:
import glob
from collections import Counter
from PIL import Image
import pytesseract
import argparse
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
import re
from imutils import contours

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'  # your path may be different

count = 1
for i in glob.glob(
        "path*.jpg"):
    print("New image")
    im = cv2.imread(i)
    shape = im.shape
    print(shape)
    # im = cv2.resize(im, dsize =(100, 100), interpolation = cv2.INTER_AREA)
    # im = cv2.resize(im, None, fx = 2, fy = 2, interpolation = cv2.INTER_CUBIC)
    if (im.shape[0] < 50) or (im.shape[1] < 50):
        print("Original image has been resized to 100*100")
        im = cv2.resize(im, dsize =(100, 100), interpolation = cv2.INTER_AREA)
    im3 = im.copy()
    gray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray,(5,5),0)
    thresh = cv2.adaptiveThreshold(blur,255,1,1,11,2)
    
    #################      Now finding Contours         ###################
    cont,hierarchy = cv2.findContours(thresh,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
    # sorted_ctrs = sorted(cont, key=lambda ctr: cv2.boundingRect(ctr)[0])
    '''if parameter is 0, it will sort it on the basis of x axis, 
    else if its 1 then y axis.'''
    # ltr = sorted(cont, key=lambda ctr: cv2.boundingRect(ctr)[0])
    # ltr = sorted(cont, key=lambda ctr: cv2.boundingRect(ctr)
                  # [0] + cv2.boundingRect(ctr)[1] * im.shape[1] )
    ltr = contours.sort_contours(cont, method="left-to-right")[0]#''
    area_list = []
    coord_left= []
    coord_right=[]
    mylist = None
    for cnt in ltr:
        area =  cv2.contourArea(cnt)
        if cv2.contourArea(cnt)>200:
            [x,y,w,h] = cv2.boundingRect(cnt)
            if mylist is None:
                if h>25 and h<70 and w <50:
                    area_list.append(area)
                    mylist = [{
                    'x':x,
                    'y':y,
                    'w':w,
                    'h':h
                }]                 
            else:                    
                if h>25 and h<70 and w <50:
                    mylist.append({'x':x, 'y':y,'w':w, 'h':h})
                    area_list.append(area)
                    print(h)
    if mylist:                
        unique_list = [i for n, i in enumerate(mylist) if i not in mylist[n + 1:]] 
        # break
        for i in unique_list:        
            x = i['x']
            y = i['y']
            w = i['w']
            h = i['h']        
            # cv2.rectangle(im,(x,y),(x+w,y+h),(255,255,255),0)
            # print(x,y,w,h)
            left_top = (x , y)
            right_bottom = (x+w , y+h)
            left_bottom = (x+w , y)
            right_top = (x , y+h)        
            # print("the type iss",type(left_top))
            # print(left_top ,
            #       right_bottom,
            #       left_bottom , 
            #       right_top)
            coord_left.append(left_top)
            coord_right.append(right_bottom)
            print(coord_left)
            # image = cv2.circle(im, (left_top[0],left_top[1]), radius=1, color=(0, 0, 255), thickness=5)
            # image = cv2.circle(im, (right_top[0],right_top[1]), radius=1, color=(255, 0, 0), thickness=5)
            # image = cv2.circle(im, (right_bottom[0],right_bottom[1]), radius=1, color=( 0, 0,255), thickness=5)
            # image = cv2.circle(im, (left_bottom[0],left_bottom[1]), radius=1, color=( 255,0, 0), thickness=5)
            # roi = thresh[y:y+h,x:x+w]
            # roismall = cv2.resize(roi,(10,10))
            # count += 1
            roi = im[y-1:y+h,x-2:x+w+2]
            # cv2.imshow('norm',im)
            # cv2.waitKey(0)
            # roi_resize = cv2.resize(roi, dsize =(50, 50), interpolation = cv2.INTER_AREA)
            # cv2.imwrite('./crop/'+str(count)+'.jpg', roi_resize)
            print("Images are being saved")
                # break
            # cv2.imshow('norm',im)
            # cv2.waitKey(0)
            # cv2.destroyAllWindows()  
    # if len(coord_left)>1:
    if mylist:    
        textleft = coord_left[0]
        textbottom = coord_right[-1]
        x1 = textleft[0]-10
        y1 = textleft[1]-15
        x2 = textbottom[0]+10
        y2 = textbottom[1]+20
        # cv2.rectangle(im,(x1,y1),(x2,y2),(0,255,0),2)
        # cv2.imshow('norm',im)
        # cv2.waitKey(0)
        
        posList= ((x1,y1),(x2,y1-10),(x1-15,y2),(x2-15,y2-10))
        print("Image is being sent for transformation")
        print(posList)
        # specify desired output size 
        width = im.shape[1]
        height = im.shape[0]
        # specify conjugate x,y coordinates (not y,x)
        input1 = np.float32([posList[0], posList[1],posList[2],posList[3]]) #ww323.jpg
        output = np.float32([[0,0], [width,0], [0,height], [width,height]])
        # compute perspective matrix
        matrix = cv2.getPerspectiveTransform(input1,output)
        #do perspective transformation setting area outside input to black
        imgOutput = cv2.warpPerspective(im, matrix, (width,height))#, cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=(0,0,0)
        print(imgOutput.shape)
        # save the warped output
        cv2.imwrite("path"+str(count)+".jpg", imgOutput)
        count += 1
        print("End")
        # cv2.imwrite("PNP_deskewed\\deskewed"+i, imgOutput)
        # show the result
        # cv2.imshow("result", imgOutput)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows() 


lis = []
new= []
for i in glob.glob("path*.jpg"):
#i = '8.jpg'
    print("-------------------- new image ------------------")
    img = cv2.imread(i,0)
    print("Original image size",img.shape)
    img1 = cv2.resize(img, None, fx=6, fy=4)
    bimg = cv2.GaussianBlur(img1,(3,3),0)
    ret,thresh = cv2.threshold(bimg, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    sharpen_filter = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
    sharped_img = cv2.filter2D(thresh, -1, sharpen_filter)
    # Testing all PSM values
    for i in range(6,14):
        #print(f'PSM: [{i}]',pytesseract.image_to_string(sharped_img, 
          #                                config = f'--psm {i} --oem 3 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'))
        text = pytesseract.image_to_string(sharped_img, 
                                      config = f'--psm {i} --oem 3 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789')    
        lis.append(text)
        if lis[0] == '\x0c':
            lis = []    
        else:
            new.append(lis[0].split('\n')[0])   
            lis = []
        plt.subplot(1,1,1),plt.imshow(sharped_img, cmap = 'gray')
        plt.xticks([]),plt.yticks([])
    
    counts = Counter(new).most_common()
    print(counts)
    if len(counts) > 1:
        print("THE EXTRACTED TEXT IS :",counts[0][0])
        new = []

        plt.show()
            #plt.imshow(sharped_img, cmap= 'gray');
        length = len(counts[0][0])
    #print(length)
        if length < 7:
            if length % 2 == 0:
                a_list = list(counts[0][0][0:int(length/2)])
                n_list = list(counts[0][0][int(length/2):])
            else:
                length += 1
                length = int(length/2)
                print(length)
                a_list = list(counts[0][0][0:length])
                n_list = list(counts[0][0][(length)-1:])

        else:    
            num = re.search("[A-Z]{2,4}[0-9]{2,3}",counts[0][0])  
            numerals = re.search("[0-9]{4}", counts[0][0]) 
            if num is not None:
                num = num[0]
            if numerals is not None:
                num_length = len(numerals[0])
                numerals = numerals[0]
                print("NUMERALS CASE CHECK ",numerals)
                if length % 2 == 0:
                    a_list = list(num[0:int(length/2)])
                    n_list = list(numerals[int(length/2):])
                else:
                    length += 1
                    length = int(length/2)
                    a_list = list(num[0:length-1])
                    n_list = list(numerals[-4:])
            elif num is not None:
                #print(num)
                if length % 2 == 0:
                    a_list = list(num[0:int(length/2)])
                    n_list = list(num[int(length/2):])
                else:
                    length += 1
                    length = int(length/2)
                    print(length)
                    a_list = list(num[0:length])
                    n_list = list(num[-3:])    
            #a_list = a_list[-3:]
        #print(a_list)
        #print(n_list)
        ALPHA_LIST = ['A' if i=='4' else 
                   'B' if i=='8' else 
                   'D' if i=='0' else
                   'I' if i=='1' else 
                   'T' if i=='9' else 
                   'S' if i=='5' else
                   'Z' if i== '2' else 
                      i for i in a_list] 
        NUM_LIST = ['4' if i=='A' else 
                   '8' if i=='B' else 
                   '0' if i=='D' else
                   '1' if i=='I' else
                   '9' if i=='T' else 
                   '5' if i=='S' else 
                   '2' if i== 'Z' else i for i in n_list] 
        #print(ALPHA_LIST)
        #print(NUM_LIST)
        if n_list[0] != NUM_LIST[0]:
            ALPHA_LIST = ALPHA_LIST[0:3]
        else:
            ALPHA_LIST = ALPHA_LIST[-3:]    

        FULL_LIST = ALPHA_LIST+NUM_LIST    
        FULL_LIST = ''.join(FULL_LIST)
        
        print("POST PROCESSED EXTRACTED TEXT :",FULL_LIST)   
        FULL_LIST = []